<a href="https://colab.research.google.com/github/JiihwanKim/Quantum_Information_Processing/blob/master/MinErrorDiscrimination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minimum Error Discrimination


In [0]:
%reset -f

 MinErrorDiscrimination Calculates guessing probability and optimals
 
 This function has three required arguments:
 
    q: prior probability list (1xN)
    
    RHO: density operators(dxd array)'s list (1xN)
    
    precision: number (1x1)
    
 Pg,M,K = MinErrorDiscrimination(q,RHO,precision) calculates guessing
 
 probability(Pg), optimal measurement(M) and symmetry operator(K).
 
 Requires: cvxpy, numpy
 
 Author: Jihwan Kim (mslab.k@gmail.com )
 
 Affiliation: Mathematical Science Lab, Applied Physics, Hanyang University, South Korea
 
 Last updated: Jun. 21, 2019

In [0]:
# Minimum Error Discrimination from Dual Problem

def MinErrorDiscriminationDual(q,RHO,precision):
  if isinstance(q,list) and isinstance(RHO,list) and len(q)==len(RHO):
    import cvxpy as cp
    num_of_states = len(RHO)
    
    # Construction of the problem.
    K = cp.Variable((len(RHO[1]),len(RHO[1])),symmetric=True);
    objective = cp.Minimize(cp.trace(K));
    constraints = [K>>q[x]*RHO[x] for x in range(num_of_states)];
    prob = cp.Problem(objective, constraints);
    prob.solve()
    
    # Return the optimals.
    K = np.around(K.value,precision)
    Pg = np.trace(K)
    M = [np.around(constraints[x].dual_value,precision) 
         for x in range(num_of_states)];
    return  Pg, M, K
  
  else: raise ValueError("Unknown format")

In [0]:
# Minimum Error Discrimination from Primal Problem

def MinErrorDiscrimination(q,RHO,precision):
  if isinstance(q,list) and isinstance(RHO,list) and len(q)==len(RHO):
    def Correct_Probability(q,RHO,M):
      return cp.trace(cp.sum([q[x]*RHO[x]*M[x] for x in range(num_of_states)]))
    import cvxpy as cp
    num_of_states = len(RHO)
    # Construction of the problem.
    M = [cp.Variable((len(RHO[0]),len(RHO[0])),symmetric=True) for x in range(num_of_states)];
    objective = cp.Maximize(Correct_Probability(q,RHO,M));
    constraints = [M[x]>>0 for x in range(num_of_states)]+[cp.sum(M)==np.eye(len(RHO[0]))];
    prob = cp.Problem(objective, constraints);
    prob.solve()
    
#   Return the optimals.
    K = np.around(constraints[-1].dual_value,precision)
    Pg = objective.value
    M = [np.around(M[x].value,precision) for x in range(len(M))]
    return Pg,M,K
  
  else: raise ValueError("Unknown format")


# Example
Suppose that the given states are prepared in a system with equiprobably.
<p align="center">
$\rho_0 = \pmatrix{1&0\\ 0&0}$,&nbsp;&nbsp;&nbsp;  $\rho_1 = \frac{1}{2}\pmatrix{1&1\\ 1&1}$,&nbsp;&nbsp;&nbsp; $\rho_2 = \frac{1}{2}\pmatrix{1&0\\ 0&1}$
</p>
Find a optimal measurement providing guessing probability.

In [4]:
import numpy as np
q = [1/3,1/3,1/3];
RHO = [np.array([[1, 0],[0, 0]])]
RHO.append(np.array([[1, 1],[1, 1]])*0.5)
RHO.append(np.array([[1, 0],[0, 1]])*0.5)
Pg,M,K = MinErrorDiscrimination(q,RHO,3)
print(M)

[array([[ 0.854, -0.354],
       [-0.354,  0.146]]), array([[0.146, 0.354],
       [0.354, 0.854]]), array([[-0.,  0.],
       [ 0., -0.]])]


In [5]:
print('Guessing Probability: '+str(Pg)+'\n')
print('Optimal Measurement:\n')
print('\n'.join(['M'+str(x)+'\n='+str(M[x]) for x in range(len(RHO))]))
print('\nDual Optimal: '+'\n')
print('K\n='+str(K))

Guessing Probability: 0.5690355936134054

Optimal Measurement:

M0
=[[ 0.854 -0.354]
 [-0.354  0.146]]
M1
=[[0.146 0.354]
 [0.354 0.854]]
M2
=[[-0.  0.]
 [ 0. -0.]]

Dual Optimal: 

K
=[[0.368 0.083]
 [0.083 0.201]]
